In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob 
import os
import plotly.graph_objects as go
import plotly.express as px
import plotly
import pathlib
plotly.io.orca.config.executable = str(pathlib.Path().absolute()) + '/bins/orca'
print(plotly.io.orca.config.executable)

In [ ]:
filenames = list(glob.iglob("../OUTPUT/Sink/*.csv"))
print(len(filenames))

In [ ]:
pd_dataframes = {'continuous': [], 'phi': [], 'summary': [], 'positions': [], 'initialpositions': [], 'finished': [], 'lost': []}
for filename in filenames:
    full_path = os.path.abspath(filename)
    path = os.path.dirname(full_path)
    basename = os.path.basename(filename)
    finished_filename = path + "/" + "finished_" + "_".join(basename.split("_")[1:])
    #if not os.path.isfile(finished_filename):
    #    print("%s does not exist ! " % os.path.basename(finished_filename))
    #    print("%s does exist" % basename)
    #    continue
    try:
        data_file_delimiter = ','
        header = []
        with open(full_path, 'r') as temp_f:
            header = temp_f.readline().rstrip('\n').split(data_file_delimiter) + ['empty']
        df = pd.read_csv(full_path, names=header, skiprows=1)
        basename = os.path.basename(filename)
        name = basename.split("_")[0]
        arguments = basename.rstrip(".csv").split("_")[3:]
        it = iter(arguments)
        for val in it:
            parameter = val.split("=")[0]
            value = val.split("=")[1]
            df[parameter] = value
        pd_dataframes[name].append(df)
    except Exception as e:
        print(e)
        #print(full_path)
        print("Error: "+ full_path)

In [ ]:
dfo = {}
for name in pd_dataframes:
    if(pd_dataframes[name]):
        dfo[name] = pd.concat(pd_dataframes[name])
df = dfo.copy()

In [ ]:
def sourceAddressToAgentId(sa):
    try:
        return int(sa.split(".")[-1])-1
    except AttributeError:
        print(sa)
        return -1

df['continuous']['agent_id'] = df['continuous']['Source Address'].apply(sourceAddressToAgentId)
df['summary']['agent_id'] = df['summary']['Source Address'].apply(sourceAddressToAgentId)
df['continuous']['Throughput'] = (df['continuous']['Rx Bytes']*8)

In [ ]:
common_columns = ['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'antennaGain', 'dataRate', 'udp', 'RngRun', 'scenarioType', 'agent_id']
df['continuous'] = df['continuous'].set_index(common_columns)
df['positions'] = df['positions'].set_index(common_columns)
df['summary'] = df['summary'].set_index(common_columns)

In [ ]:
df['continuous'].head()

In [ ]:
df['positions'].head()

In [ ]:
df['summary'].head()

In [ ]:
new_df = df['continuous'].join(df['positions'], how='left', on=common_columns, lsuffix='_left', rsuffix='_right')
common_columns += ['Source Address', 'Destination Address']
new_df = new_df.reset_index().set_index(common_columns)
df['summary'] = df['summary'].reset_index().set_index(common_columns)
new_df = new_df.join(df['summary'], how='left', on=common_columns, rsuffix='_total')
new_df = new_df.reset_index()

In [ ]:
new_df.head()

In [ ]:
convergence_time = []
g = None
for name, group in new_df.groupby(['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'antennaGain', 'dataRate', 'udp', 'RngRun']):
    local_data = []
    antenna_convergence = group['last_change'].max()
    throughput_convergence = None
    full_rx = None
    g = group
    for agent_id, agent_group in group.groupby(['agent_id']):
        low, high = 0.95*(agent_group['Rx Bytes'].values[-1]), 1.05*(agent_group['Rx Bytes'].values[-1])
        agent_group['between'] = agent_group['Rx Bytes'].between(low, high)
        agent_group['between'] = ~agent_group['between']
        k = agent_group['between'][agent_group['between']].index.max()
        #k = agent_group['between'][agent_group['between']==False].last_valid_index()
        if (k is None) or (pd.isna(k)):
            throughput_convergence = 0
        else:
            throughput_convergence = agent_group.loc[k]['Time']
        if name[4] == "Drone":
            full_rx = agent_group[agent_group['Time'] >= antenna_convergence]['Throughput'].mean()
        else:
            full_rx = agent_group['Throughput'].mean()
        convergence_time.append(list(name) + [agent_id, throughput_convergence, antenna_convergence, full_rx])

In [ ]:
conv = pd.DataFrame(convergence_time, columns=['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'antennaGain', 'dataRate', 'udp', 'RngRun', 'agent_id', 'Throughput Converge', 'Antenna Converge', 'Throughput'])
conv = conv.groupby(['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'antennaGain', 'dataRate', 'udp', 'RngRun']).mean().reset_index()
full = conv.copy()
conv = conv[conv['agentType'] == 'Drone']

In [ ]:
radius = '100'
filtered = conv[(conv['distance'] == radius) & (conv['antennaGain'] == "3.0")]
filtered = filtered[(filtered['dataRate'] == '50Mbps')]
filtered = filtered[(filtered['N'] == '5')]

trace0 = go.Box(
    y=filtered['Antenna Converge'],
    x=filtered['wifiManager'],
    name='Orientation Convergence Time',
    boxmean=True
)

trace1 = go.Box(
    y=filtered['Throughput Converge'],
    x=filtered['wifiManager'],
    name='Throughput Convergence Time',
    boxmean=True
)

data = [trace0, trace1]
layout = go.Layout(
    #title = "foo",
    template = "simple_white",
    xaxis_title="Wifi Manager",
    yaxis=go.layout.YAxis(
        title='Time (s)',
        range=[-5, max(filtered['Throughput Converge'].max()*1.05, filtered['Antenna Converge'].max()*1.05)],
        zeroline=False
    ),
    boxmode='group'
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend_orientation="h")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.show()
fig.write_image("convergence_sink_3dB.png")

In [ ]:
radius = '100'
filtered = conv[(conv['distance'] == radius) & (conv['antennaGain'] == "3.0")]
print(set(filtered['N']))
filtered = filtered[(filtered['N'] == '5')]
filtered = filtered[(filtered['dataRate'] == '50Mbps')]
drone = filtered[(filtered['agentType'] == 'Drone')]
client = filtered[(filtered['agentType'] == 'Client')]

trace0 = go.Box(
    y=(client['Throughput']),
    x=client['wifiManager'],
    name='Client',
    boxmean=True
)

trace1 = go.Box(
    y=(drone['Throughput']),
    x=drone['wifiManager'],
    name='Drone',
    boxmean=True
)


data = [trace0, trace1]
layout = go.Layout(
    template = "simple_white",
    xaxis_title="Wifi Manager",
    yaxis=go.layout.YAxis(
        title='Throughput (bps)',
    ),
    boxmode='group'
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend_orientation="h")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.show()
fig.write_image("débit_sink_3dB.png")

In [ ]:
full[full['Throughput'] <= 1000]

In [ ]:
full[full['agentType'] == "Drone"].head()

In [ ]:
df['summary'] = df['summary'].reset_index()
df['summary']['Throughput'] = (df['summary']['Rx Bytes']*8) / pd.to_numeric(df['summary']['simulationTime'])
mads = []
for name, group in df['summary'].groupby(['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'dataRate', 'udp', 'RngRun']):
    jain = ((group['Throughput'].sum())**2)/((len(group['Throughput'].index))*(group['Throughput']*group['Throughput']).sum())
    mads.append(list(name) + [group['Throughput'].mean(), group['Throughput'].mad(), jain])
mads = pd.DataFrame(mads, columns=['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'dataRate', 'udp', 'RngRun', 'Throughput', 'MAD', 'Jain'])

In [ ]:
a = mads.copy()
a['distance'] = pd.to_numeric(a['distance'])
a['dataRate'] = pd.to_numeric(a['dataRate'].str.rstrip("Mbps"))
a['N'] = pd.to_numeric(a['N'])
a = a[a['distance'] == 100]
a = a[a['N'] == 5]
a.sort_values(by=['wifiManager', 'dataRate', 'agentType', 'distance'], inplace=True)
a['cross'] = a['wifiManager'] + " " + a['dataRate'].apply(lambda x: str(x))

drone = a[(a['agentType'] == 'Drone')]
client = a[(a['agentType'] == 'Client')]

trace0 = go.Box(
    y=(client['Jain']),
    x=client['wifiManager'],
    name='Client',
    boxmean=True
)

trace1 = go.Box(
    y=(drone['Jain']),
    x=drone['wifiManager'],
    name='Drone',
    boxmean=True
)


data = [trace0, trace1]
layout = go.Layout(
    template = "simple_white",
    xaxis_title="Wifi Manager",
    yaxis=go.layout.YAxis(
        title='Jain index',
    ),
    boxmode='group'
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend_orientation="h")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.show()

In [ ]:
radius = '100'
filtered = full[(full['distance'] == radius)]
filtered = filtered[(filtered['N'] == '5')]
filtered['cross'] = filtered['wifiManager'] + " " + filtered['dataRate'].apply(lambda x: str('{num:03d}'.format(num=int(x.rstrip("Mbps")))))
filtered = filtered.sort_values(by=['cross'])
drone = filtered[(filtered['agentType'] == 'Drone')]
client = filtered[(filtered['agentType'] == 'Client')]


fig = go.Figure(data=data, layout=layout)

trace0 = go.Box(
    y=(client['Throughput']),
    x=client['cross'],
    name='Isotropic Antenna',
    boxmean=True
)

trace1 = go.Box(
    y=(drone['Throughput']),
    x=drone['cross'],
    name='AP-AC-Mesh Antenna',
    boxmean=True
)


data = [trace0, trace1]
layout = go.Layout(
    template = "simple_white",
    xaxis_title="Wifi Manager",
    yaxis=go.layout.YAxis(
        title='Throughput (bps)',
    ),
    boxmode='group'
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend_orientation="h")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.show()
fig.write_image("100m_sink_comparison_3dB.png")

In [ ]:
set(mads['distance'])

In [ ]:
pd.options.display.float_format = "{:.2f}".format
mads['N'] = pd.to_numeric(mads['N'])
a=mads.copy()
a=a[a['dataRate'] == '50Mbps']
a.groupby(['N', 'distance', 'wifiManager', 'dataRate', 'simulationTime', 'agentType']).describe()['Jain'].reset_index()